In [2]:
# Import Data Science Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import itertools
import random

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
import seaborn as sns

# Tensorflow Libraries
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import Adam

# System libraries
from pathlib import Path
import os.path

# Metrics
from sklearn.metrics import classification_report, confusion_matrix

sns.set_style('darkgrid')

In [3]:
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

In [4]:
dataset = "/kaggle/input/animal-image-dataset-90-different-animals"
def convert_path_to_df(dataset):
    image_dir = Path(dataset)

    # Get filepaths and labels
    filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.jpeg')) + list(image_dir.glob(r'**/*.PNG'))

    labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    image_df = pd.concat([filepaths, labels], axis=1)
    return image_df

image_df = convert_path_to_df(dataset)

In [5]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

path = Path(dataset).rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

In [6]:
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=42)
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
)
# Split the data into three categories.
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 3456 validated image filenames belonging to 90 classes.
Found 864 validated image filenames belonging to 90 classes.
Found 1080 validated image filenames belonging to 90 classes.


In [7]:
augment = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224,224),
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.1),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.experimental.preprocessing.RandomContrast(0.1),
])

In [8]:
pretrained_model = tf.keras.applications.vgg16.VGG16(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

pretrained_model.trainable = False

58889256/58889256 [==============================] - 3s 0us/step


In [10]:
checkpoint_path = "animals_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               min_delta=0.01,
                               patience = 4,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [11]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import create_tensorboard_callback

--2023-04-25 05:12:50--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-04-25 05:12:50 (50.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [12]:
inputs = pretrained_model.input
x = augment(inputs)

x = Dense(128, activation='relu')(pretrained_model.output)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)

outputs = Dense(90, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=Adam(0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=200,
    callbacks=[
        early_stopping,
        create_tensorboard_callback("training_logs", 
                                    "animals_classification"),
        checkpoint_callback,
        #reduce_lr
    ]
)

model.save("AnimalRecognizer.h5")

Saving TensorBoard log files to: training_logs/animals_classification/20230425-051256
Epoch 1/200
108/108 [==============================] - 71s 557ms/step - loss: 5.7990 - accuracy: 0.0107 - val_loss: 5.0254 - val_accuracy: 0.0150
Epoch 2/200
108/108 [==============================] - 59s 548ms/step - loss: 5.7822 - accuracy: 0.0119 - val_loss: 4.9126 - val_accuracy: 0.0127
Epoch 3/200
108/108 [==============================] - 59s 549ms/step - loss: 5.7332 - accuracy: 0.0116 - val_loss: 4.8959 - val_accuracy: 0.0127
Epoch 4/200
108/108 [==============================] - 59s 551ms/step - loss: 5.6688 - accuracy: 0.0142 - val_loss: 4.8701 - val_accuracy: 0.0150
Epoch 5/200
108/108 [==============================] - 60s 552ms/step - loss: 5.6338 - accuracy: 0.0142 - val_loss: 4.8444 - val_accuracy: 0.0139
Epoch 6/200
108/108 [==============================] - 59s 543ms/step - loss: 5.5748 - accuracy: 0.0150 - val_loss: 4.8071 - val_accuracy: 0.0162
Epoch 7/200
108/108 [=================

In [13]:
#for more epochs
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=200,
    callbacks=[
        early_stopping,
        create_tensorboard_callback("training_logs", 
                                    "animals_classification"),
        checkpoint_callback,
        #reduce_lr
    ]
)

Saving TensorBoard log files to: training_logs/animals_classification/20230425-083001
Epoch 1/200
108/108 [==============================] - 61s 567ms/step - loss: 2.0654 - accuracy: 0.4708 - val_loss: 1.7359 - val_accuracy: 0.6331
Epoch 2/200
108/108 [==============================] - 58s 537ms/step - loss: 2.0735 - accuracy: 0.4583 - val_loss: 1.7315 - val_accuracy: 0.6354
Epoch 3/200
108/108 [==============================] - 58s 536ms/step - loss: 2.0405 - accuracy: 0.4618 - val_loss: 1.7220 - val_accuracy: 0.6354
Epoch 4/200
108/108 [==============================] - 58s 539ms/step - loss: 2.0574 - accuracy: 0.4685 - val_loss: 1.7171 - val_accuracy: 0.6366
Epoch 5/200
108/108 [==============================] - 58s 537ms/step - loss: 2.0681 - accuracy: 0.4656 - val_loss: 1.7093 - val_accuracy: 0.6354
Epoch 6/200
108/108 [==============================] - 58s 540ms/step - loss: 2.0548 - accuracy: 0.4688 - val_loss: 1.7017 - val_accuracy: 0.6366
Epoch 7/200
108/108 [=================

In [14]:
model.save("AnimalRecogniser1.h5")